In [3]:
    from importnb import Interactive
    with Interactive(display=True):
        from install import repos
    from poser import *

In [9]:
kernel_name = 'bench'

    !source activate $kernel_name && pip install importnb timeout_decorator poser pyyaml

In [6]:
env = λ.ChainMap()()

In [7]:
for directory in λ.map(a.str.rsplit('/', 1).last().Path())(repos):
    if (directory / 'requirements.txt').exists():
        env = λ.Path.read_text().str.splitlines()[lambda x: {'dependencies': [
            {'pip': x}
        ]}][{directory: λ.identity()}][env.new_child](directory / 'requirements.txt')
    if (directory / 'environment.yml').exists():
        env = λ.Path.read_text().io.StringIO().yaml.safe_load()[{directory: λ.identity()}][env.new_child](directory / 'environment.yml')

In [14]:
    dependencies = (
        λ / λ.valmap(λ.get('dependencies')).dict.values()
    ).concat().list()[lambda x: sum(x, [])](env.maps)

    dependencies = (
        λ
        .filter(λ**dict)
        .map(x['pip'])
        .concat().map(a.str.split('==').first()).unique().list()
        [[λ[{'pip': λ.identity()}]]]
    ).concatv(
        (λ//λ**str*a.unique().sorted().list())(dependencies)
    ).list()[{
        'name': kernel_name,
        'channels': (
            λ.map(dict.values)
            .concat().map(λ.methodcaller('get', 'channels', [])).concat().unique().sorted().list()
        )(env.maps),
        'dependencies': λ.identity(),
    }](dependencies)
    dependencies['dependencies'][-1]['pip'] = list(set(dependencies['dependencies'][-1]['pip']) - set(dependencies['dependencies'][:-1]))

    condadeps = ['astropy', 'importnb', 'tqdm', 'ipywidgets']
    pipdeps = ['poser']

    dependencies['dependencies'] = condadeps + dependencies['dependencies']
    dependencies['dependencies'][-1]['pip'] += pipdeps

In [16]:
def write():
    with open('environment.yml', 'w') as f:
        λ.yaml.safe_dump(default_flow_style=False)[f.write](dependencies)